In [1]:
import argparse
import gc
import json
import math
import os
import random
import time
import warnings
from functools import partial
from itertools import chain
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import torch
import torch.nn.functional as F
from datasets import Dataset, features
from hydra import compose, initialize
from matplotlib import pyplot as plt
from omegaconf import OmegaConf
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, _document_frequency
from sklearn.metrics import cohen_kappa_score, f1_score, log_loss, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.utils.validation import check_is_fitted
from src.seed import seed_everything
from tokenizers import AddedToken
from torch import nn
from torch.utils.data.dataloader import DataLoader

# from torch.utils.data.dataset import Dataset
from tqdm.auto import tqdm
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

warnings.filterwarnings("ignore")


with initialize(config_path="config", version_base=None):
    cfg = compose(config_name="config")
    cfg.exp_number = Path().resolve().name

print(OmegaConf.to_yaml(cfg, resolve=True))

seed_everything(cfg.seed)


bert:
  params:
    model_path: microsoft/deberta-v3-large
    metric: auc
    target_col_class_num: 2
    max_length: 192
    fp16: true
    learning_rate: 2.0e-05
    epochs: 2
    per_device_train_batch_size: 8
    per_device_eval_batch_size: 32
    steps: 50
    lr_scheduler_type: linear
    weight_decay: 0.01
exp_number: '010'
run_name: base
data:
  data_root: ../../data
  results_root: ../../results
  train_path: ../../data/train.csv
  cloth_path: ../../data/clothing_master.csv
  test_path: ../../data/test.csv
  sample_submission_path: ../../data/sample_submission.csv
  results_dir: ../../results/010/base
seed: 42
n_splits: 5
target: Recommended IND



### データの準備


In [9]:
train_df = pl.read_csv(cfg.data.train_path, try_parse_dates=True)
test_df = pl.read_csv(cfg.data.test_path, try_parse_dates=True)

skf = StratifiedKFold(n_splits=cfg.n_splits, shuffle=True, random_state=cfg.seed)


In [10]:
DEBUG = False

if DEBUG:
    train_df = train_df.head(100)


def preprocess_text(df: pl.DataFrame) -> pl.DataFrame:
    # df = df.with_columns(pl.col("Title").fill_null("none"), pl.col("Review Text").fill_null("none"))
    df = df.with_columns(
        pl.concat_str(
            [
                pl.lit("TITLE: "),
                pl.col("Title").fill_null("none"),
                pl.lit(" [SEP] "),
                pl.lit(" Review Text: "),
                pl.col("Review Text").fill_null("none"),
            ]
        ).alias("prompt")
    )
    return df


train_df = preprocess_text(train_df)
test_df = preprocess_text(test_df)

train_df = train_df.with_columns(pl.col(cfg.target).cast(pl.Int8).alias("labels"))


In [11]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(cfg.bert.params.model_path)


def tokenize(sample):
    return tokenizer(sample["prompt"], max_length=cfg.bert.params.max_length, truncation=True)


In [12]:
# token長を確認 --> max_length 180とかで大丈夫そう
print(
    f"train_df: {train_df.select(pl.col('prompt').map_elements(lambda x: len(tokenizer(x)['input_ids']))).max().item()}"
)
print(
    f"test_df: {test_df.select(pl.col('prompt').map_elements(lambda x: len(tokenizer(x)['input_ids']))).max().item()}"
)


train_df: 167
test_df: 166


### 学習

In [6]:
# metricをAUCに変更
def compute_metrics(p):
    preds, labels = p
    preds = torch.softmax(torch.tensor(preds), dim=1).numpy()
    score = roc_auc_score(labels, preds[:, 1])
    return {"auc": score}


# 実験結果格納用のディレクトリを作成
cfg.run_name = time.strftime("%Y%m%d_%H%M%S")
Path(cfg.data.results_dir).mkdir(exist_ok=True, parents=True)

y_train = train_df[cfg.target].to_numpy()
oof = np.zeros(len(y_train))

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, y_train)):
    ds_train = Dataset.from_pandas(train_df[train_idx][["prompt", "labels"]].clone().to_pandas())
    ds_val = Dataset.from_pandas(train_df[val_idx][["prompt", "labels"]].clone().to_pandas())

    config = AutoConfig.from_pretrained(cfg.bert.params.model_path)
    model = AutoModelForSequenceClassification.from_pretrained(cfg.bert.params.model_path, config=config)

    ds_train = ds_train.map(tokenize).remove_columns("prompt")
    ds_val = ds_val.map(tokenize).remove_columns("prompt")

    output_dir = os.path.join(cfg.data.results_dir, f"fold{fold}")

    train_args = TrainingArguments(
        output_dir=output_dir,
        fp16=cfg.bert.params.fp16,
        learning_rate=cfg.bert.params.learning_rate,
        num_train_epochs=cfg.bert.params.epochs,
        per_device_train_batch_size=cfg.bert.params.per_device_train_batch_size,
        per_device_eval_batch_size=cfg.bert.params.per_device_eval_batch_size,
        gradient_accumulation_steps=4,
        report_to="none",
        evaluation_strategy="steps",
        do_eval=True,
        eval_steps=cfg.bert.params.steps,
        save_total_limit=1,
        save_strategy="steps",
        save_steps=cfg.bert.params.steps,
        logging_steps=cfg.bert.params.steps,
        load_best_model_at_end=True,
        lr_scheduler_type=cfg.bert.params.lr_scheduler_type,
        metric_for_best_model=cfg.bert.params.metric,
        greater_is_better=True,
        warmup_ratio=0.1,
        weight_decay=cfg.bert.params.weight_decay,
        save_safetensors=True,
        seed=cfg.seed,
        data_seed=cfg.seed,
    )

    trainer = Trainer(
        model=model,
        args=train_args,
        train_dataset=ds_train,
        eval_dataset=ds_val,
        data_collator=DataCollatorWithPadding(tokenizer),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    final_output_dir = f"{cfg.data.results_dir}/fold{fold}/final"
    trainer.save_model(final_output_dir)
    tokenizer.save_pretrained(final_output_dir)

    pred = torch.softmax(torch.tensor(trainer.predict(ds_val).predictions), dim=1).numpy()
    oof[val_idx] = pred[:, 1]


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Auc
50,0.529300,0.325165,0.934540
100,0.258000,0.274278,0.939150
150,0.215800,0.189721,0.966245
200,0.194900,0.165006,0.970294
250,0.191100,0.159131,0.972479
300,0.149800,0.167920,0.972851
350,0.117500,0.184824,0.970660
400,0.165800,0.184622,0.972124
450,0.133800,0.174326,0.971082
500,0.171100,0.156883,0.974071


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss,Validation Loss,Auc
50,0.472100,0.255531,0.934770
100,0.220500,0.305584,0.953515
150,0.195500,0.213473,0.961886
200,0.201900,0.225082,0.960428
250,0.179200,0.197166,0.967191
300,0.126700,0.259606,0.965244
350,0.150000,0.194886,0.965475
400,0.112200,0.184723,0.967551
450,0.120800,0.209220,0.968461
500,0.164500,0.190707,0.969514


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Auc
50,0.463400,0.634845,0.782657
100,0.295400,0.282309,0.955719
150,0.265800,0.184776,0.969339
200,0.204100,0.170777,0.971697
250,0.205000,0.187462,0.970605
300,0.155900,0.176990,0.971264
350,0.141000,0.181597,0.973622
400,0.147900,0.176934,0.973421
450,0.152700,0.172399,0.972062
500,0.148600,0.172886,0.972388


### Sub


In [31]:
preds = []
for fold in range(cfg.n_splits):
    # ベストステップのモデルを取得
    fold_dir = f"{cfg.data.results_dir}/fold{fold}"
    checkpoint_dirs = [d for d in os.listdir(fold_dir) if d.startswith("checkpoint-")]
    results_dir = os.path.join(fold_dir, checkpoint_dirs[0])
    model = AutoModelForSequenceClassification.from_pretrained(results_dir)

    ds_test = Dataset.from_pandas(test_df.select("prompt").clone().to_pandas())
    ds_test = ds_test.map(tokenize).remove_columns("prompt")

    test_args = TrainingArguments(
        output_dir=cfg.data.results_dir,
        per_device_eval_batch_size=cfg.bert.params.per_device_eval_batch_size,
        do_predict=True,
        dataloader_drop_last=False,
    )

    trainer = Trainer(
        model=model,
        args=test_args,
        data_collator=DataCollatorWithPadding(tokenizer),
        tokenizer=tokenizer,
    )
    predictions = torch.softmax(torch.tensor(trainer.predict(ds_test).predictions), dim=1).numpy()
    preds.append(predictions[:, 1])

pred = np.mean(preds, axis=0)


Map:   0%|          | 0/11155 [00:00<?, ? examples/s]

Map:   0%|          | 0/11155 [00:00<?, ? examples/s]

Map:   0%|          | 0/11155 [00:00<?, ? examples/s]

Map:   0%|          | 0/11155 [00:00<?, ? examples/s]

Map:   0%|          | 0/11155 [00:00<?, ? examples/s]

In [32]:
# 提出
sub_df = pl.read_csv(cfg.data.sample_submission_path)
sub_df = sub_df.with_columns(pl.Series(pred).alias("target"))
sub_df.write_csv(os.path.join(cfg.data.results_dir, f"{cfg.run_name}_submission.csv"))
sub_df.head()


target
f32
0.998845
0.141182
0.998687
0.085266
0.996718
